In [58]:
import os
import torch

os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.1.0+cu118
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [59]:
import argparse
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from math import sqrt
import math
from statistics import mean

from torch_geometric.datasets import TUDataset

import torch as th
import torch
import torch.nn as nn
from torch import Tensor
from torch.nn.parameter import Parameter
import torch_geometric
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.nn import Linear, LayerNorm
from sklearn import metrics
from scipy.spatial.distance import hamming
import statistics
import pandas
from time import perf_counter
from IPython.core.display import deepcopy
from torch_geometric.nn import MessagePassing
import copy
from torch.nn import ReLU, Sequential
from torch import sigmoid
from itertools import chain
from time import perf_counter
from torch_geometric.data import Data, Batch, Dataset
from functools import partial
from torch_geometric.utils import to_networkx
from torch_geometric.utils import remove_self_loops
from torch_geometric.utils import k_hop_subgraph
from typing import Callable, Union, Optional
#from torch_geometric.utils.num_nodes import maybe_num_nodes
import networkx as nx
from typing import List, Tuple, Dict
from collections import Counter
import statistics
import csv
from torch_geometric.utils import to_undirected

from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader
import torch_geometric.nn as gnn

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
Explainability_name = 'SubGraphX'
Task_name = 'Node Classification'
Model_Name = "GCN"
checkpoint_directory_Classifier = "/content/drive/My Drive/Node Classification on Uncertain Graph/" + str(Explainability_name) + " on " + str(Task_name) + "/Model/model_classifier.pt"
classifier_lr = 0.001
classifier_dropout = 0.1
classifier_weight_decay = 1e-6
classifier_bias = True
Model_name = "GCN"
DataSet_name = "DBLP"

File_Name = Model_name + " " + Explainability_name + " " + Task_name + " " + DataSet_name + " "

In [5]:

pathname = "/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/"
authors_name_2_ID = np.load(pathname + "authors_name_2_ID.npy", allow_pickle=True).item()
authors_ID_2_name = inv_map = {v: k for k, v in authors_name_2_ID.items()}


pathname = "/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/"
authorID_2_nodefeatures_and_label = np.load(pathname + "authorID_2_nodefeatures_and_label.npy", allow_pickle=True).item()

data_x = torch.load('/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/data_x.pt')
data_x = data_x.float()



zhang_id = authors_name_2_ID['Ce Zhang 0001']
barna_id = authors_name_2_ID['Barna Saha']
margo_id = authors_name_2_ID['Margo I. Seltzer']
yizhou_id = authors_name_2_ID['Yizhou Sun']



Zhang_index = list(authorID_2_nodefeatures_and_label.keys()).index(zhang_id)
Barna_index = list(authorID_2_nodefeatures_and_label.keys()).index(barna_id)
Margo_index = list(authorID_2_nodefeatures_and_label.keys()).index(margo_id)
Yizhou_index = list(authorID_2_nodefeatures_and_label.keys()).index(yizhou_id)

In [6]:
class GCN_model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN_model, self).__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(160, hidden_channels, bias=True)
        self.conv2 = GCNConv(hidden_channels, hidden_channels, bias=True)
        self.conv3 = GCNConv(hidden_channels, hidden_channels, bias=True)
        self.lin1 = Linear(hidden_channels, hidden_channels, bias=True)
        self.lin2 = Linear(hidden_channels, 8, bias=True)


        self.act_f = nn.Softmax(dim=1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        x = self.act_f(x)
        return x


In [7]:
number_of_possible_worlds = 10
def get_probabilities_for_subgraphs_by_different_possible_worlds(person_index):
    person_predictions_by_different_possible_worlds = dict()
    for possible_world_index in range(10):
        GNN_Model = GCN_model(hidden_channels=100)
        checkpoint = torch.load("/content/drive/My Drive/Node Classification on Uncertain Graph/" + str(Explainability_name) + " on " + str(Task_name) + "/Model/output_possible_world"+str(possible_world_index)+".pt", map_location=torch.device('cpu'))
        GNN_Model.load_state_dict(checkpoint['model_state_dict'])
        edge_index = torch.load('/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/edge_index_T' + str(possible_world_index) + '.pt', map_location=torch.device('cpu'))


        person_predictions_by_different_possible_worlds[possible_world_index] = GNN_Model(data_x, edge_index)[person_index].tolist()

    return person_predictions_by_different_possible_worlds

In [8]:
margo_preds_on_10_pws = get_probabilities_for_subgraphs_by_different_possible_worlds(Margo_index)

In [9]:
torch.save(margo_preds_on_10_pws, '/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/margo_preds_on_10_pws.pt')

In [11]:
zhang_preds_on_10_pws = get_probabilities_for_subgraphs_by_different_possible_worlds(Zhang_index)

In [12]:
torch.save(zhang_preds_on_10_pws, '/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/zhang_preds_on_10_pws.pt')

In [13]:
barna_preds_on_10_pws = get_probabilities_for_subgraphs_by_different_possible_worlds(Barna_index)

In [14]:
torch.save(barna_preds_on_10_pws, '/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/barna_preds_on_10_pws.pt')

In [15]:
yizhou_preds_on_10_pws = get_probabilities_for_subgraphs_by_different_possible_worlds(Yizhou_index)

In [16]:
torch.save(yizhou_preds_on_10_pws, '/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/yizhou_preds_on_10_pws.pt')

In [7]:
margo_preds_on_10_pws = torch.load('/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/margo_preds_on_10_pws.pt')
zhang_preds_on_10_pws = torch.load('/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/zhang_preds_on_10_pws.pt')
barna_preds_on_10_pws = torch.load('/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/barna_preds_on_10_pws.pt')
yizhou_preds_on_10_pws = torch.load('/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/yizhou_preds_on_10_pws.pt')

In [12]:
#########             AI


yizhou_top_preds_index = []
for key, pred in yizhou_preds_on_10_pws.items():
    print(key, pred)
    internal_max = pred.index(max(pred))
    if internal_max == 0:
        yizhou_top_preds_index.append(key)
print(yizhou_top_preds_index)

0 [0.8264297842979431, 0.003987893462181091, 0.0035417918115854263, 0.05119040608406067, 0.03286315128207207, 0.01570710726082325, 0.06427454203367233, 0.0020052443724125624]
1 [0.9806742668151855, 0.00021990739332977682, 0.00046913977712392807, 0.011805977672338486, 0.0011365094687789679, 0.0007371263345703483, 0.004297369159758091, 0.0006597127649001777]
2 [0.9907354712486267, 0.0001381316687911749, 0.0018731012241914868, 0.0016304808668792248, 0.00042275365558452904, 0.0010405423818156123, 0.003963425289839506, 0.00019611528841778636]
3 [0.9987279772758484, 3.5784137253358494e-06, 4.5110718929208815e-06, 0.0012173071736469865, 9.646084436099045e-06, 2.674569941518712e-06, 3.3844105928437784e-05, 4.212408555304137e-07]
4 [0.14440611004829407, 0.008941645734012127, 0.002629841910675168, 0.025490092113614082, 0.03540579229593277, 0.16630388796329498, 0.5609013438224792, 0.0559212900698185]
5 [0.9146870970726013, 0.005024902988225222, 0.004073689691722393, 0.02716878429055214, 0.0111570

In [13]:
#########             Alg = 5


barna_top_preds_index = []
for key, pred in barna_preds_on_10_pws.items():
    print(key, pred)
    internal_max = pred.index(max(pred))
    if internal_max == 5:
        barna_top_preds_index.append(key)
print(barna_top_preds_index)

0 [0.14625784754753113, 0.011525291949510574, 0.013265264220535755, 0.06496547907590866, 0.07317718863487244, 0.18701601028442383, 0.5009601712226868, 0.0028327498584985733]
1 [0.36769044399261475, 0.011218481697142124, 0.007329646032303572, 0.0739876925945282, 0.030365707352757454, 0.4398551285266876, 0.053322162479162216, 0.01623075269162655]
2 [0.10423639416694641, 0.004453601781278849, 0.004719585180282593, 0.008445863611996174, 0.009718552231788635, 0.7361556887626648, 0.13065996766090393, 0.0016104108653962612]
3 [0.015347085893154144, 0.0020056653302162886, 0.00031383472378365695, 0.001198817859403789, 0.008538666181266308, 0.9478810429573059, 0.024088621139526367, 0.0006261367816478014]
4 [0.028913594782352448, 0.002966376719996333, 0.00041078423964791, 0.004924855660647154, 0.0093397106975317, 0.9147385358810425, 0.01916738785803318, 0.01953878626227379]
5 [0.11730793118476868, 0.008788448758423328, 0.009464186616241932, 0.025235489010810852, 0.008666160516440868, 0.7589212656

In [14]:
#########             AI


zhang_top_preds_index = []
for key, pred in zhang_preds_on_10_pws.items():
    print(key, pred)
    internal_max = pred.index(max(pred))
    if internal_max == 0:
        zhang_top_preds_index.append(key)
print(zhang_top_preds_index)

0 [0.45524388551712036, 0.015689192339777946, 0.006172188092023134, 0.08727581799030304, 0.3080679178237915, 0.044842272996902466, 0.07307610660791397, 0.009632540866732597]
1 [0.8429896235466003, 0.005941294599324465, 0.009985481388866901, 0.04637876898050308, 0.032327406108379364, 0.012794184498488903, 0.026374822482466698, 0.023208433762192726]
2 [0.9491422176361084, 0.001313885091803968, 0.007455204613506794, 0.008295592851936817, 0.004515423905104399, 0.00610609445720911, 0.021875843405723572, 0.0012956543359905481]
3 [0.9339491724967957, 0.002593001816421747, 0.006540146190673113, 0.02913804166018963, 0.010130166076123714, 0.004454128444194794, 0.01211040560156107, 0.0010849005775526166]
4 [0.22213199734687805, 0.020251503214240074, 0.01393156312406063, 0.0815601572394371, 0.03538067266345024, 0.1604137122631073, 0.3054022789001465, 0.16092801094055176]
5 [0.9598556160926819, 0.002788506681099534, 0.0025685448199510574, 0.02091996930539608, 0.005422111134976149, 0.003443402005359

In [15]:
#########             AI                      & SW


margo_top_preds_index = []
for key, pred in margo_preds_on_10_pws.items():
    print(key, pred)
    internal_max = pred.index(max(pred))
    if internal_max == 0:
        margo_top_preds_index.append(key)
print(margo_top_preds_index)

0 [0.11895874887704849, 0.011522969231009483, 0.0018323323456570506, 0.04498295858502388, 0.7599929571151733, 0.025718525052070618, 0.033439066261053085, 0.003552376991137862]
1 [0.39152082800865173, 0.02844885364174843, 0.04491845518350601, 0.06971897184848785, 0.14102892577648163, 0.05351926386356354, 0.07798995822668076, 0.19285468757152557]
2 [0.20414616167545319, 0.0347718819975853, 0.08818113803863525, 0.10428483784198761, 0.0981510803103447, 0.05406077206134796, 0.38183772563934326, 0.03456632420420647]
3 [0.11790554225444794, 0.030078040435910225, 0.14492511749267578, 0.028271324932575226, 0.47070595622062683, 0.09372790902853012, 0.09814387559890747, 0.016242220997810364]
4 [0.14534874260425568, 0.01945800893008709, 0.015103451907634735, 0.031783025711774826, 0.027107594534754753, 0.31800854206085205, 0.10239391028881073, 0.3407967984676361]
5 [0.6553001403808594, 0.018642935901880264, 0.014820287004113197, 0.2184252291917801, 0.04140857979655266, 0.020504795014858246, 0.02245

In [8]:
barna_subgraphs = torch.load('/content/drive/My Drive/Node Classification on Uncertain Graph/SubGraphX on Node Classification/Barna Important_Subgraphs_by_name_and_ID.pt')
margo_subgraphs = torch.load('/content/drive/My Drive/Node Classification on Uncertain Graph/SubGraphX on Node Classification/Margo Important_Subgraphs_by_name_and_ID.pt')
yizhou_subgraphs = torch.load('/content/drive/My Drive/Node Classification on Uncertain Graph/SubGraphX on Node Classification/Yizhou Important_Subgraphs_by_name_and_ID.pt')
zhang_subgraphs = torch.load('/content/drive/My Drive/Node Classification on Uncertain Graph/SubGraphX on Node Classification/Zhang Important_Subgraphs_by_name_and_ID.pt')

In [10]:
barna_candidate_subgraphs = dict()
margo_candidate_subgraphs = dict()
yizhou_candidate_subgraphs = dict()
zhang_candidate_subgraphs = dict()

In [16]:
for i in margo_top_preds_index:
    margo_candidate_subgraphs[i] = margo_subgraphs[i]

for i in barna_top_preds_index:
    barna_candidate_subgraphs[i] = barna_subgraphs[i]

for i in yizhou_top_preds_index:
    yizhou_candidate_subgraphs[i] = yizhou_subgraphs[i]

for i in zhang_top_preds_index:
    zhang_candidate_subgraphs[i] = zhang_subgraphs[i]

In [17]:
print(margo_candidate_subgraphs)
print(barna_candidate_subgraphs)
print(yizhou_candidate_subgraphs)
print(zhang_candidate_subgraphs)

{1: {'Important_CoauthorsIDs': [123, 9164, 22299, 23901, 57169, 58456, 65561, 69990, 70320, 71441, 75247, 75301, 75302, 75394, 76028, 77880, 79713, 97898, 108775, 113061, 138342, 257674, 297674, 301400, 307953, 308006, 308007, 372442, 385362, 385366, 396534, 401429, 401454, 401807, 543507, 592175, 622390], 'Important_CoauthorsNames': ['Cynthia Rudin', 'Hongyu Yang', 'res', 'Rui Zhang', 'Margo I. Seltzer', 'Jonathan Appavoo', 'Stijn Vansummeren', 'Thomas Moyer', 'Daniel W. Margo', 'Jonathan Ledlie', 'David Mazi', 'Adam Bates 0001', 'Olivier Hermant', 'Rui Xin', 'Nate Foster', 'Peter R. Pietzuch', 'Yuanyuan Zhou 0001', 'Matt Welsh', 'Mema Roussopoulos', 'Jeffrey Shneidman', 'David M. Brooks', 'Ryan P. Adams', 'John K. Ousterhout', 'Daniel Alabi', 'Yasuhiro Endo', 'Stephen Manley', 'Michael Courage', 'Krzysztof Z. Gajos', 'Hayden McTavish', 'Jacques Chen', 'Paul Gardner', 'James Gwertzman', 'Antonio Dias', 'Mark Miller', 'Maia L. Jacobs', 'Satoshi Asami', 'Michael J. Tucker']}, 5: {'Impor

In [9]:
edges_source_target_prob = torch.load('/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/edges_source_target_prob.pt')

In [18]:
margo_candidate_subgraphs_probs = dict()
for key, values in margo_candidate_subgraphs.items():
    prob_list = []
    subgraph_node_ids = values['Important_CoauthorsIDs']
    for pairs, prob in edges_source_target_prob.items():
        (a1, a2) = pairs
        if (a1 in subgraph_node_ids) and (a2 in subgraph_node_ids):
            prob_list.append(prob)
    margo_candidate_subgraphs_probs[key] = prob_list

In [19]:
print(margo_candidate_subgraphs_probs)

{1: [0.6321205588285577, 0.6321205588285577, 0.9999996940976795, 0.6321205588285577, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.9179150013761012, 0.6321205588285577, 0.7768698398515702, 0.7768698398515702, 0.6321205588285577, 0.7768698398515702, 0.6321205588285577, 0.8646647167633873, 0.6321205588285577, 0.6321205588285577, 0.6321205588285577, 0.3934693402873666, 0.6321205588285577, 0.6321205588285577, 0.9179150013761012, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.6321205588285577, 0.3934693402873666, 0.3934693402873666, 0.9179150013761012, 0.6321205588285577, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873

In [20]:
barna_candidate_subgraphs_probs = dict()
for key, values in barna_candidate_subgraphs.items():
    prob_list = []
    subgraph_node_ids = values['Important_CoauthorsIDs']
    for pairs, prob in edges_source_target_prob.items():
        (a1, a2) = pairs
        if (a1 in subgraph_node_ids) and (a2 in subgraph_node_ids):
            prob_list.append(prob)
    barna_candidate_subgraphs_probs[key] = prob_list

In [21]:
yizhou_candidate_subgraphs_probs = dict()
for key, values in yizhou_candidate_subgraphs.items():
    prob_list = []
    subgraph_node_ids = values['Important_CoauthorsIDs']
    for pairs, prob in edges_source_target_prob.items():
        (a1, a2) = pairs
        if (a1 in subgraph_node_ids) and (a2 in subgraph_node_ids):
            prob_list.append(prob)
    yizhou_candidate_subgraphs_probs[key] = prob_list

In [22]:
zhang_candidate_subgraphs_probs = dict()
for key, values in zhang_candidate_subgraphs.items():
    prob_list = []
    subgraph_node_ids = values['Important_CoauthorsIDs']
    for pairs, prob in edges_source_target_prob.items():
        (a1, a2) = pairs
        if (a1 in subgraph_node_ids) and (a2 in subgraph_node_ids):
            prob_list.append(prob)
    zhang_candidate_subgraphs_probs[key] = prob_list

In [23]:
print(zhang_candidate_subgraphs_probs)

{0: [0.8646647167633873, 0.9179150013761012, 0.9999938557876467, 0.9888910034617577, 0.995913228561536, 0.3934693402873666, 0.8646647167633873, 0.3934693402873666, 0.6321205588285577, 0.6321205588285577, 0.9932620530009145, 0.7768698398515702, 0.7768698398515702, 0.9698026165776815, 0.7768698398515702, 0.9932620530009145, 0.9816843611112658, 0.7768698398515702, 0.950212931632136, 0.9698026165776815, 0.9999546000702375, 0.950212931632136, 0.9999546000702375, 0.9816843611112658, 0.9997965316309894, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.6321205588285577, 0.3934693402873666, 0.9984965608070224, 0.3934693402873666, 0.6321205588285577, 0.9179150013761012, 0.6321205588285577, 0.6321205588285577, 0.9179150013761012, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.3934693402873666, 0.6321205588285577

In [24]:
margo_candidate_subgraphs_entropies = dict()
for key, probs in margo_candidate_subgraphs_probs.items():
    ent_list = []
    for prob in probs:
        ent_list.append(-(prob*math.log2(prob) + (1-prob)*math.log2(1-prob)))
    margo_candidate_subgraphs_entropies[key] = sum(ent_list)/2

In [25]:
barna_candidate_subgraphs_entropies = dict()
for key, probs in barna_candidate_subgraphs_probs.items():
    ent_list = []
    for prob in probs:
        ent_list.append(-(prob*math.log2(prob) + (1-prob)*math.log2(1-prob)))
    barna_candidate_subgraphs_entropies[key] = sum(ent_list)/2

In [26]:
yizhou_candidate_subgraphs_entropies = dict()
for key, probs in yizhou_candidate_subgraphs_probs.items():
    ent_list = []
    for prob in probs:
        ent_list.append(-(prob*math.log2(prob) + (1-prob)*math.log2(1-prob)))
    yizhou_candidate_subgraphs_entropies[key] = sum(ent_list)/2

In [27]:
zhang_candidate_subgraphs_entropies = dict()
for key, probs in zhang_candidate_subgraphs_probs.items():
    ent_list = []
    for prob in probs:
        ent_list.append(-(prob*math.log2(prob) + (1-prob)*math.log2(1-prob)))
    zhang_candidate_subgraphs_entropies[key] = sum(ent_list)/2

In [28]:
print(margo_candidate_subgraphs_entropies)
print(barna_candidate_subgraphs_entropies)
print(yizhou_candidate_subgraphs_entropies)
print(zhang_candidate_subgraphs_entropies)

{1: 34.28175523402673, 5: 23.29775299074821, 8: 20.898220078497435}
{1: 21.666262947988784, 2: 14.487823341922663, 3: 30.862946007239476, 4: 18.252453696075754, 5: 14.210910498748813, 6: 26.17695146243664}
{0: 224.92863854502139, 1: 235.23763124578494, 2: 254.27295177876857, 3: 247.3310672793793, 5: 220.75378799195724, 6: 222.99790561160697, 8: 196.61867667853178, 9: 239.98954152618703}
{0: 238.8164225563009, 1: 223.30879783693416, 2: 139.22093329195144, 3: 234.98476678991005, 5: 245.32652269464276, 6: 220.26183773690568, 8: 255.91278031320684, 9: 256.6228265062557}


In [29]:
import operator

In [30]:
margo_candidate_subgraphs_entropies_sorted = dict(sorted(margo_candidate_subgraphs_entropies.items(), key=operator.itemgetter(1), reverse=True))

In [31]:
barna_candidate_subgraphs_entropies_sorted = dict(sorted(barna_candidate_subgraphs_entropies.items(), key=operator.itemgetter(1), reverse=True))

In [32]:
yizhou_candidate_subgraphs_entropies_sorted = dict(sorted(yizhou_candidate_subgraphs_entropies.items(), key=operator.itemgetter(1), reverse=True))

In [33]:
zhang_candidate_subgraphs_entropies_sorted = dict(sorted(zhang_candidate_subgraphs_entropies.items(), key=operator.itemgetter(1), reverse=True))

In [34]:
print(margo_candidate_subgraphs_entropies_sorted)
print(barna_candidate_subgraphs_entropies_sorted)
print(yizhou_candidate_subgraphs_entropies_sorted)
print(zhang_candidate_subgraphs_entropies_sorted)

{1: 34.28175523402673, 5: 23.29775299074821, 8: 20.898220078497435}
{3: 30.862946007239476, 6: 26.17695146243664, 1: 21.666262947988784, 4: 18.252453696075754, 2: 14.487823341922663, 5: 14.210910498748813}
{2: 254.27295177876857, 3: 247.3310672793793, 9: 239.98954152618703, 1: 235.23763124578494, 0: 224.92863854502139, 6: 222.99790561160697, 5: 220.75378799195724, 8: 196.61867667853178}
{9: 256.6228265062557, 8: 255.91278031320684, 5: 245.32652269464276, 0: 238.8164225563009, 3: 234.98476678991005, 1: 223.30879783693416, 6: 220.26183773690568, 2: 139.22093329195144}


In [35]:
top_r = 4
margo_candidate_subgraphs_topR_ents = list(margo_candidate_subgraphs_entropies_sorted.keys())[:top_r]
barna_candidate_subgraphs_topR_ents = list(barna_candidate_subgraphs_entropies_sorted.keys())[:top_r]
yizhou_candidate_subgraphs_topR_ents = list(yizhou_candidate_subgraphs_entropies_sorted.keys())[:top_r]
zhang_candidate_subgraphs_topR_ents = list(zhang_candidate_subgraphs_entropies_sorted.keys())[:top_r]

In [36]:
print(margo_candidate_subgraphs_topR_ents)
print(barna_candidate_subgraphs_topR_ents)
print(yizhou_candidate_subgraphs_topR_ents)
print(zhang_candidate_subgraphs_topR_ents)

[1, 5, 8]
[3, 6, 1, 4]
[2, 3, 9, 1]
[9, 8, 5, 0]


In [37]:
margo_topR_subgraphs_nodes = dict()
for subgraph_index in margo_candidate_subgraphs_topR_ents:
    margo_topR_subgraphs_nodes[subgraph_index] = margo_candidate_subgraphs[subgraph_index]['Important_CoauthorsIDs']

In [38]:
print(margo_topR_subgraphs_nodes)

{1: [123, 9164, 22299, 23901, 57169, 58456, 65561, 69990, 70320, 71441, 75247, 75301, 75302, 75394, 76028, 77880, 79713, 97898, 108775, 113061, 138342, 257674, 297674, 301400, 307953, 308006, 308007, 372442, 385362, 385366, 396534, 401429, 401454, 401807, 543507, 592175, 622390], 5: [123, 22299, 48728, 57169, 58456, 70319, 71441, 75302, 75304, 77880, 85249, 97898, 110988, 112655, 113061, 113158, 197793, 297674, 307429, 307953, 308006, 372442, 375672, 385364, 385366, 401454, 401807, 543507, 592175, 595717], 8: [123, 48728, 57169, 65561, 70319, 71441, 75300, 75303, 75394, 77327, 77880, 79713, 97898, 110988, 112655, 138342, 197793, 257674, 297674, 308006, 372442, 385363, 401292, 401453, 401454, 543507, 592176, 595717]}


In [39]:
barna_topR_subgraphs_nodes = dict()
for subgraph_index in barna_candidate_subgraphs_topR_ents:
    barna_topR_subgraphs_nodes[subgraph_index] = barna_candidate_subgraphs[subgraph_index]['Important_CoauthorsIDs']

In [40]:
yizhou_topR_subgraphs_nodes = dict()
for subgraph_index in yizhou_candidate_subgraphs_topR_ents:
    yizhou_topR_subgraphs_nodes[subgraph_index] = yizhou_candidate_subgraphs[subgraph_index]['Important_CoauthorsIDs']

In [41]:
zhang_topR_subgraphs_nodes = dict()
for subgraph_index in zhang_candidate_subgraphs_topR_ents:
    zhang_topR_subgraphs_nodes[subgraph_index] = zhang_candidate_subgraphs[subgraph_index]['Important_CoauthorsIDs']

In [42]:
print(margo_topR_subgraphs_nodes)
print(barna_topR_subgraphs_nodes)
print(yizhou_topR_subgraphs_nodes)
print(zhang_topR_subgraphs_nodes)

{1: [123, 9164, 22299, 23901, 57169, 58456, 65561, 69990, 70320, 71441, 75247, 75301, 75302, 75394, 76028, 77880, 79713, 97898, 108775, 113061, 138342, 257674, 297674, 301400, 307953, 308006, 308007, 372442, 385362, 385366, 396534, 401429, 401454, 401807, 543507, 592175, 622390], 5: [123, 22299, 48728, 57169, 58456, 70319, 71441, 75302, 75304, 77880, 85249, 97898, 110988, 112655, 113061, 113158, 197793, 297674, 307429, 307953, 308006, 372442, 375672, 385364, 385366, 401454, 401807, 543507, 592175, 595717], 8: [123, 48728, 57169, 65561, 70319, 71441, 75300, 75303, 75394, 77327, 77880, 79713, 97898, 110988, 112655, 138342, 197793, 257674, 297674, 308006, 372442, 385363, 401292, 401453, 401454, 543507, 592176, 595717]}
{3: [4896, 44468, 48401, 48405, 49048, 49441, 50436, 53601, 54587, 59558, 63899, 66159, 67891, 69708, 94900, 101232, 101252, 101558, 101597, 101705, 104748, 105606, 105624, 105985, 106336, 107231, 107845, 118215, 216314, 217927, 217975, 226112, 226742], 6: [4896, 30430, 481

In [43]:
margo_topR_subgraphs_edges = [[],[]]
for key, values in margo_topR_subgraphs_nodes.items():

    for pairs, prob in edges_source_target_prob.items():
        (a1, a2) = pairs
        if (a1 in values) and (a2 in values):
            margo_topR_subgraphs_edges[0].append(a1)
            margo_topR_subgraphs_edges[1].append(a2)
margo_topR_subgraphs_edge_index = to_undirected(torch.tensor(margo_topR_subgraphs_edges))

In [44]:
print(margo_topR_subgraphs_edge_index)

tensor([[   123,    123,    123,    123,    123,    123,    123,    123,    123,
            123,   9164,   9164,  22299,  22299,  22299,  22299,  22299,  22299,
          23901,  23901,  23901,  48728,  48728,  48728,  48728,  57169,  57169,
          57169,  57169,  57169,  57169,  57169,  57169,  57169,  57169,  57169,
          57169,  57169,  57169,  57169,  57169,  57169,  57169,  57169,  57169,
          57169,  57169,  57169,  57169,  57169,  57169,  57169,  57169,  57169,
          57169,  57169,  57169,  57169,  57169,  57169,  57169,  57169,  57169,
          57169,  57169,  57169,  57169,  57169,  57169,  57169,  57169,  57169,
          57169,  57169,  57169,  57169,  57169,  57169,  57169,  57169,  58456,
          58456,  58456,  65561,  65561,  65561,  69990,  69990,  69990,  70319,
          70319,  70320,  71441,  71441,  71441,  71441,  71441,  71441,  75247,
          75247,  75247,  75247,  75247,  75247,  75300,  75300,  75301,  75301,
          75301,  75302,  75

In [45]:
barna_topR_subgraphs_edges = [[],[]]
for key, values in barna_topR_subgraphs_nodes.items():

    for pairs, prob in edges_source_target_prob.items():
        (a1, a2) = pairs
        if (a1 in values) and (a2 in values):
            barna_topR_subgraphs_edges[0].append(a1)
            barna_topR_subgraphs_edges[1].append(a2)
barna_topR_subgraphs_edge_index = to_undirected(torch.tensor(barna_topR_subgraphs_edges))

In [57]:
print(len(barna_topR_subgraphs_edge_index[0])/2)

124.0


In [46]:
yizhou_topR_subgraphs_edges = [[],[]]
for key, values in yizhou_topR_subgraphs_nodes.items():

    for pairs, prob in edges_source_target_prob.items():
        (a1, a2) = pairs
        if (a1 in values) and (a2 in values):
            yizhou_topR_subgraphs_edges[0].append(a1)
            yizhou_topR_subgraphs_edges[1].append(a2)
yizhou_topR_subgraphs_edge_index = to_undirected(torch.tensor(yizhou_topR_subgraphs_edges))

In [47]:
zhang_topR_subgraphs_edges = [[],[]]
for key, values in zhang_topR_subgraphs_nodes.items():

    for pairs, prob in edges_source_target_prob.items():
        (a1, a2) = pairs
        if (a1 in values) and (a2 in values):
            zhang_topR_subgraphs_edges[0].append(a1)
            zhang_topR_subgraphs_edges[1].append(a2)
zhang_topR_subgraphs_edge_index = to_undirected(torch.tensor(zhang_topR_subgraphs_edges))

In [62]:
clean_list = [[], []]

In [68]:
def update_possible_worlds(person_name, topR_subgraphs_edge_index):
    global clean_list
    GNN_Model = GCN_model(hidden_channels=100)
    for possible_world_index in range(10):
        checkpoint = torch.load("/content/drive/My Drive/Node Classification on Uncertain Graph/" + str(Explainability_name) + " on " + str(Task_name) + "/Model/output_possible_world"+str(possible_world_index)+".pt", map_location=torch.device('cpu'))
        GNN_Model.load_state_dict(checkpoint['model_state_dict'])
        edge_index = torch.load('/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/edge_index_T' + str(possible_world_index) + '.pt', map_location=torch.device('cpu'))
        edge_index = edge_index.tolist()
        #topR_subgraphs_edge_index = topR_subgraphs_edge_index.tolist()
        for i in range(len(topR_subgraphs_edge_index[0])):
            if (topR_subgraphs_edge_index[0][i] in edge_index[0]) and (edge_index[1][edge_index[0].index(topR_subgraphs_edge_index[0][i])] == topR_subgraphs_edge_index[1][i]):
                continue
            else:
                edge_index[0].append(topR_subgraphs_edge_index[0][i])
                edge_index[1].append(topR_subgraphs_edge_index[1][i])

                clean_list[0].append(topR_subgraphs_edge_index[0][i])
                clean_list[1].append(topR_subgraphs_edge_index[1][i])

        updated_edge_index = to_undirected(torch.tensor(edge_index))
        #print(updated_edge_index)
        #torch.save(updated_edge_index, '/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/'+str(person_name)+'_updated_edge_index_pw'+str(possible_world_index)+'.pt')


In [53]:
update_possible_worlds("Margo", margo_topR_subgraphs_edge_index.tolist())

In [69]:
update_possible_worlds("Barna", barna_topR_subgraphs_edge_index.tolist())

In [55]:
update_possible_worlds("Yizhou", yizhou_topR_subgraphs_edge_index.tolist())

In [56]:
update_possible_worlds("Zhang", zhang_topR_subgraphs_edge_index.tolist())

In [64]:
print(authors_name_2_ID['Barna Saha'])
barna_id = authors_name_2_ID['Barna Saha']

48401


In [70]:
print(clean_list)

[[4896, 4896, 30430, 30430, 30430, 44468, 48160, 48160, 48160, 48160, 48160, 48253, 48253, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48401, 48405, 48405, 48405, 48480, 48480, 48480, 48486, 48486, 48486, 48486, 48877, 48877, 48877, 49048, 49048, 49048, 49048, 49283, 49283, 49283, 49283, 49355, 49355, 49355, 49355, 49355, 49355, 49441, 49441, 49441, 49441, 49441, 50029, 50029, 50029, 50029, 50436, 50436, 50436, 50436, 50436, 50637, 50637, 50637, 53601, 53601, 53601, 53601, 53601, 53601, 53601, 53601, 54587, 54587, 54587, 59558, 59558, 59558, 59558, 59558, 59558, 59558, 63632, 63632, 63899, 63899, 63899, 63899, 63899, 63899, 63899, 65696, 65696, 66159, 66159, 66159, 66159, 67891, 67891, 67891, 67891, 69708,

In [66]:
pathname = "/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/"
authors_adj_list = np.load(pathname + "authors_adj_list.npy", allow_pickle=True).item()

In [67]:
print(authors_adj_list[barna_id])

[48400, 48401, 44468, 101597, 101558, 94900, 54533, 49048, 48480, 65696, 50637, 107231, 101252, 50436, 101232, 104748, 107605, 49355, 59558, 63899, 67891, 53601, 30430, 63632, 105844, 118214, 54587, 118215, 50029, 48877, 226112, 48160, 48486, 49441, 105624, 105985, 226742, 217927, 48405, 49283, 106336, 98383, 260614, 107845, 217975, 4896, 66159, 69708, 48253, 258747, 105606, 216314, 66569, 101705, 109403, 67832, 216839]


In [72]:
print(len(clean_list))
print(authorID_2_nodefeatures_and_label[0])

2
{'node_features': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'label': 6}


In [75]:
ids = []
for id in clean_list[0]:
    if authorID_2_nodefeatures_and_label[id]['label'] == 3:
        ids.append(id)

In [76]:
print(ids)

[98383, 98383, 104748, 104748, 104748, 104748, 217927, 217927, 217927, 217927, 98383, 98383, 104748, 104748, 104748, 104748, 217927, 217927, 217927, 98383, 98383, 104748, 104748, 104748, 104748, 217927, 217927, 217927, 217927, 98383, 98383, 104748, 104748, 104748, 104748, 217927, 217927, 217927, 98383, 98383, 104748, 104748, 104748, 104748, 217927, 217927, 217927, 98383, 98383, 104748, 104748, 104748, 104748, 217927, 217927, 217927, 217927, 98383, 98383, 104748, 104748, 104748, 104748, 217927, 217927, 217927, 217927, 98383, 98383, 104748, 104748, 104748, 104748, 217927, 217927, 217927, 98383, 98383, 104748, 104748, 104748, 104748, 217927, 217927, 217927, 98383, 98383, 104748, 104748, 104748, 104748, 217927, 217927, 217927, 217927]


In [78]:
idss = []
for id in ids:
    if authorID_2_nodefeatures_and_label[id]['label'] == 4:
        idss.append(id)

In [79]:
print(idss)

[]


In [80]:
pathname = "/content/drive/My Drive/Node Classification on Uncertain Graph/Dataset/"
authors_adj_list = np.load(pathname + "authors_adj_list.npy", allow_pickle=True).item()

In [81]:
print(authors_adj_list[barna_id])

[48400, 48401, 44468, 101597, 101558, 94900, 54533, 49048, 48480, 65696, 50637, 107231, 101252, 50436, 101232, 104748, 107605, 49355, 59558, 63899, 67891, 53601, 30430, 63632, 105844, 118214, 54587, 118215, 50029, 48877, 226112, 48160, 48486, 49441, 105624, 105985, 226742, 217927, 48405, 49283, 106336, 98383, 260614, 107845, 217975, 4896, 66159, 69708, 48253, 258747, 105606, 216314, 66569, 101705, 109403, 67832, 216839]


In [82]:
search_list = authors_adj_list[barna_id]

In [83]:
for id in search_list:
    if authorID_2_nodefeatures_and_label[id]['label'] == 4:
        print(id)